# Draft `pyGIMLi(emg3d)`

**NEEDS**
- `pyGIMLi`
- `emg3d`
- `discretize`
- `xarray`
- `h5py`

**Current Limitations**
- Only isotropic models supported, without el. perm. and magn. perm.

An attempt at using `pyGIMLi` as an inversion framework for `emg3d` computations.

For developping purposes, we take a very simple grid/model/survey:
- Coarse mesh, no stretching (potentially too small).
- Simple double-halfspace model water-subsurface with a resistive block.
- Survey: A single 2D line, 6 sources, 1 frequency.

=> For this dev-implementation we also do inversion crime, using the same grid for forward modelling and inversion.

In [1]:
import emg3d
import numpy as np
import pygimli as pg
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm

In [2]:
%matplotlib notebook

In [3]:
pg.Report('emg3d')

--------------------------------------------------------------------------------
  Date: Tue Dec 13 10:33:25 2022 Mitteleuropäische Zeit

                OS : Windows
            CPU(s) : 8
           Machine : AMD64
      Architecture : 64bit
               RAM : 15.9 GiB
       Environment : Jupyter

  Python 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 05:59:45)
  [MSC v.1929 64 bit (AMD64)]

             emg3d : 1.8.0
           pygimli : 1.3.1
            pgcore : Module not found
             numpy : 1.19.5
        matplotlib : 3.5.1
             scipy : 1.9.1
              tqdm : 4.64.1
           IPython : 7.33.0
           pyvista : 0.34.1
--------------------------------------------------------------------------------

## Load survey (incl. data) and initial model

In [4]:
data = emg3d.load('pginv.h5')
survey = data['survey']
model = data['model']
grid = model.grid

Data loaded from «C:\Guenther.T\src\gimli\dev-pygimli-emg3d\pginv.h5»
[emg3d v1.8.0 (format 1.0) on 2022-12-13T09:41:19.885480].


In [5]:
survey = survey.select(sources='TxED-1')

## Jacobian Wrapper

In [6]:
class EMG3DJacobian(pg.Matrix):
    
    def __init__(self, sim):
        super().__init__()
        self.sim = sim

    def cols(self):
        # sim.model.size corresponds to the number of cells
        return self.sim.model.size

    def rows(self):
        # sim.survey.count corresponds to the number of non-NaN data points.
        return self.sim.survey.count * 2

    def mult(self, x):
        """J * x """
        # reshape or ravel
        jvec = self.sim.jvec(x.reshape(self.sim.model.shape, order='F'))
        data = jvec[self.sim._finite_data]
        return np.hstack((data.real, data.imag))

    def transMult(self, x):
        """J.T * x = (x * J.T)^T """
        data = np.ones(survey.data.observed.shape, dtype=self.sim.data.observed.dtype)*np.nan
        data[self.sim._finite_data] = x[:x.size//2] + 1j*x[x.size//2:]
        misfit = self.sim.misfit
        return self.sim.jtvec(data).ravel('F')
    
    def save(self, *args):
        pass

## Forward Wrapper

In [7]:
class MyFWD(pg.Modelling):
    
    def __init__(self, sim):
        
        # Should it be here or at the end of the __init__?
        super().__init__()
        
        # Store the simulation
        # (I replaced «fop» by «sim».)
        self.sim = sim
        
        # Translate discretize TensorMesh to pg-Grid
        self.mesh_ = pg.createGrid(
            x=sim.model.grid.nodes_x,
            y=sim.model.grid.nodes_y,
            z=sim.model.grid.nodes_z,
        )
        
        self.J = EMG3DJacobian(sim)
        self.setJacobian(self.J)

    def response(self, model):
        # do a lot of checks and cleanups
        
        # Clean emg3d-simulation, so things are recomputed
        self.sim.clean('computed')
        
        # Replace model
        self.sim.model = emg3d.Model(self.sim.model.grid, property_x=model)
        
        # Compute responses for new model
        self.sim.compute()
        
        # Return the responses
        data = self.sim.data.synthetic.data[self.sim._finite_data]
        return np.hstack((data.real, data.imag))
    
    def createStartModel(self, dataVals):
        return np.ones(self.sim.model.size)
    
    def createJacobian(self, model):
        pass  # do nothing

## Run an inversion

In [8]:
# Create an emg3d Simulation instance
sim = emg3d.simulations.Simulation(
    survey=survey,
    model=model,
    gridding='same',  # I will like to make that more flexible in the future
    max_workers=6,    # Adjust as needed
    receiver_interpolation='linear',  # Currently necessary for the gradient
    # solver_opts,
    tqdm_opts=False,  # Switch off verbose progress bars
    solver_opts={'plain': True, 'maxit': 1},
)

# Not sure if this is the best way, but for now it works
sim._finite_data = np.isfinite(sim.data.observed.data)

In [9]:
fop = MyFWD(sim)
fop.setMesh(fop.mesh_)
INV = pg.Inversion(fop=fop, verbose=True, debug=True)

INV.transData = pg.trans.TransSymLog(survey.noise_floor)
INV.transModel = pg.trans.TransLogLU(1, 1000)

# Regularization: Setting active/passive cells would be great,
# e.g., de-activating air and water for the inversion.
# And of course other regularizations (smoothness etc).
# INV.setRegularization(limits=[], correlationLengths=[...])
# INV.setRegularization(2, fixed=1e-14) # air
# INV.setRegularization(3, fixed=10.0) # water

dataC = sim.data.observed.data[sim._finite_data].copy()
data = np.hstack([dataC.real, dataC.imag])

In [10]:
modelVec = np.ones(fop.mesh_.cellCount())

In [11]:
respVec = fop.response(modelVec)
jresp = fop.J.transMult(respVec)
jresp.shape


= Source TxED-1; Frequency 1.0 Hz = MAX. ITERATION REACHED, NOT CONVERGED


= Source TxED-1; Frequency 1.0 Hz = MAX. ITERATION REACHED, NOT CONVERGED



(65536,)

In [12]:
jmodel = fop.J.mult(modelVec)

In [ ]:
model = INV.run(
    dataVals=data,
    verbose=True,
    # emg3d would have the standard deviation, existing of
    # relative and absolute error. Could that be provided?
    errorVals=np.ones(data.size)*sim.survey.relative_error,
)

13/12/22 - 10:35:57 - pyGIMLi - INFO - Created startmodel from forward operator: [1. 1. 1. ... 1. 1. 1.]
13/12/22 - 10:35:57 - pyGIMLi - INFO - Starting inversion.


fop: <__main__.MyFWD object at 0x000001800F5EF040>
Data transformation: <pygimli.core._pygimli_.RTrans object at 0x00000180BDF4EEE0>
Model transformation: <pygimli.core._pygimli_.RTransLog object at 0x000001800F5EF0E0>
min/max (data): -2.3e-08/1.2e-09
min/max (error): 5%/5%
min/max (start model): 1/1
--------------------------------------------------------------------------------
